In [1]:
import os
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
from models.edsr import edsr
import matplotlib as plt
import seaborn as sns
import math

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2024-09-10 20:14:48.100086: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-10 20:14:48.442296: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-10 20:14:48.442329: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-10 20:14:48.444186: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-10 20:14:48.680332: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-10 20:14:48.692674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
def quantile_clipping(data, percentage, mode="max"):
    quantile_val = np.quantile(data, percentage)
    if mode == "max":
        data = data.clip(max=quantile_val)
    if mode == "min":
        data = data.clip(min=quantile_val)
    return data

def exp_root_norm(data, exp=2):
    return data ** (1 / exp)

def minmax_scale(images):
    # Assuming images is a 4D array with shape (N, 32, 32)
    min_val = np.min(images)
    max_val = np.max(images)
    
    scaled_images = (images - min_val) / (max_val - min_val)
    
    return scaled_images

def preprocess(images):
    images = quantile_clipping(images, 0.95, mode="max")
    images = exp_root_norm(images, exp=2)
    images = minmax_scale(images)
    return images


# def pad_to_nearest_square(matrix, num_clients):
#     """
#     Pad the matrix to the nearest size that's perfectly divisible by sqrt(num_clients).
#     """
#     orig_height, orig_width = matrix.shape[1:3]
#     sqrt_clients = int(math.sqrt(num_clients))
    
#     target_size = math.ceil(max(orig_height, orig_width) / sqrt_clients) * sqrt_clients
    
#     padded = np.pad(
#         matrix,
#         ((0, 0), (0, target_size - orig_height), (0, target_size - orig_width), (0, 0)),
#         mode='constant'
#     )
    
#     return padded, (orig_height, orig_width)


In [3]:
import numpy as np
import math
from sklearn.cluster import KMeans

def extract_node_locations(dataset):
    if dataset == 'geant':
        nodes = [
            "at1.at", "be1.be", "ch1.ch", "cz1.cz", "de1.de", "es1.es", "fr1.fr", "gr1.gr",
            "hr1.hr", "hu1.hu", "ie1.ie", "il1.il", "it1.it", "lu1.lu", "nl1.nl", "ny1.ny",
            "pl1.pl", "pt1.pt", "se1.se", "si1.si", "sk1.sk", "uk1.uk"
        ]
        locations = [
            (16.3729, 48.2091), (4.3518, 50.8469), (6.1399, 46.2038), (14.4423, 50.0785),
            (8.6842, 50.1122), (-3.7033, 40.4167), (2.351, 48.8566), (23.5808, 37.9778),
            (15.9644, 45.8071), (19.0936, 47.4976), (-6.2573, 53.3416), (34.8097, 32.0714),
            (9.19, 45.4642), (6.1296, 49.6112), (4.9407, 52.3236), (-73.94384, 40.6698),
            (16.8874, 52.3963), (-9.1363, 38.7073), (17.8742, 59.3617), (14.5148, 46.0574),
            (17.1297, 48.1531), (-0.1264, 51.5086)
        ]
    elif dataset == 'germany':
        nodes = np.load("CNSM/data/germany_nodes.npy")
        locations = np.load("CNSM/data/germany_locations.npy")
        
    return np.array(nodes), np.array(locations)

# def cluster_nodes(locations, n_clusters):
#     from sklearn.cluster import KMeans
#     kmeans = KMeans(n_clusters=n_clusters, random_state=42)
#     clusters = kmeans.fit_predict(locations)
#     return clusters

def cluster_nodes(locations, num_clients):
    from sklearn.metrics import pairwise_distances
    hosts_per_cluster = len(locations)//num_clients
    num_nodes = len(locations)
    num_clusters = num_nodes // hosts_per_cluster
    
    if num_nodes % hosts_per_cluster != 0:
        raise ValueError(f"Number of nodes ({num_nodes}) is not divisible by hosts per cluster ({hosts_per_cluster})")
    
    # Initialize clusters
    clusters = np.full(num_nodes, -1)
    
    # Choose initial centroids
    centroids = [np.random.choice(num_nodes)]
    distances = pairwise_distances(locations, locations[centroids]).reshape(-1)
    
    for _ in range(1, num_clusters):
        centroids.append(np.argmax(distances))
        distances = np.minimum(distances, pairwise_distances(locations, locations[centroids[-1]].reshape(1, -1)).reshape(-1))
    
    # Assign nodes to clusters
    for cluster_id, centroid in enumerate(centroids):
        unassigned = np.where(clusters == -1)[0]
        distances_to_centroid = pairwise_distances(locations[unassigned], locations[centroid].reshape(1, -1)).reshape(-1)
        closest_nodes = unassigned[np.argsort(distances_to_centroid)[:hosts_per_cluster]]
        clusters[closest_nodes] = cluster_id
    
    return clusters

def kmeans(locations, n_clusters):
    kmeans = KMeans(n_clusters, random_state=42)
    clusters = kmeans.fit_predict(locations)
    return clusters

def pad_sub_fine_grained_matrix(fine_grained, num_hosts, c_scale_factor=2):
    _, height, width, _ = fine_grained.shape
    
    # Calculate required padding
    pad_height = math.ceil(height / c_scale_factor) * c_scale_factor - height
    pad_width = math.ceil(width / c_scale_factor) * c_scale_factor - width
    
    if pad_height == 0 and pad_width == 0:
        return fine_grained
    
    padded_fine_grained = np.pad(fine_grained, 
                                 ((0, 0), (0, pad_height), (0, pad_width), (0, 0)), 
                                 mode='constant')
    
    # Fill padding with average values
    if pad_height > 0:
        padded_fine_grained[:, -pad_height:, :, :] = np.mean(fine_grained[:, -1:, :, :], axis=(1, 2), keepdims=True)
    if pad_width > 0:
        padded_fine_grained[:, :, -pad_width:, :] = np.mean(fine_grained[:, :, -1:, :], axis=(1, 2), keepdims=True)
    
    return padded_fine_grained


def create_client_data(fine_matrices, num_clients, c_scale_factor, nodes, locations, overlap_percentage):
    print(f"Splitting data across {num_clients} clients with {overlap_percentage}% overlap...")
    num_samples, height, width, channels = fine_matrices.shape
    
    # Calculate the size of each fine-grained sub-matrix
    sub_hr_size = height // num_clients + overlap_percentage * len(nodes) // 100

    # node_grid_size = int(np.ceil(np.sqrt(len(nodes))))
    
    # Adjust sub_size to be divisible by c_scale_factor
    adj_sub_hr_size = (sub_hr_size // c_scale_factor) * c_scale_factor
    
    # Ensure sub_size is at least c_scale_factor
    adj_sub_hr_size = max(adj_sub_hr_size, c_scale_factor)
    
    # Calculate grid dimensions
    grid_size = int(np.ceil(np.sqrt(num_clients)))
    
    # Resize the fine matrices
    # resized_fine_matrices = np.zeros((num_samples, new_size, new_size, channels))
    # resized_fine_matrices[:, :min(height, new_size), :min(width, new_size), :] = fine_matrices[:, :min(height, new_size), :min(width, new_size), :]
    # resized_fine_matrices = np.zeros((num_samples, adj_sub_hr_size, adj_sub_hr_size, channels))
    # resized_fine_matrices[:, :min(height, adj_sub_hr_size), :min(width, adj_sub_hr_size), :] = fine_matrices[:, :min(height, adj_sub_hr_size), :min(width, adj_sub_hr_size), :]
    
    # Calculate the number of overlapping nodes
    num_overlap_nodes = int(len(nodes) * overlap_percentage / 100)
    # Randomly select overlapping nodes
    overlap_nodes = np.random.choice(nodes, num_overlap_nodes, replace=False)
    remaining_nodes = [node for node in nodes if node not in overlap_nodes]
    num_nodes_to_split = (len(remaining_nodes)//num_clients) * num_clients
    remaining_nodes = np.random.choice(remaining_nodes, num_nodes_to_split, replace=False)

    # Assign nodes to FL clients using K-means with fixed number of elements per cluster
    remaining_nodes_locations = [locations[nodes.tolist().index(node)] for node in remaining_nodes]
    clusters = cluster_nodes(np.array(remaining_nodes_locations), num_clients)
    client_unique_nodes = [remaining_nodes[clusters == i] for i in range(num_clients)]
    
    client_data = []
    for client in range(num_clients):
        # Select unique nodes for this client
        client_nodes = np.concatenate([overlap_nodes, client_unique_nodes[client]])
        np.random.shuffle(client_nodes)

        client_locations = [locations[nodes.tolist().index(node)] for node in client_nodes]
                
        # Create mask for selecting data for this client
        mask = np.isin(nodes, client_nodes)
        mask_2d = mask[:, np.newaxis] & mask[np.newaxis, :]
        
        # Extract data for this client
        # We add some padding here if the clients_nodes are not divisible by c_scale_factor
        client_fine = fine_matrices[:, mask_2d].reshape(num_samples, len(client_nodes), len(client_nodes), channels)
        if len(client_nodes) % c_scale_factor != 0:
            client_fine = pad_sub_fine_grained_matrix(client_fine, len(client_nodes))
        
        # Create low-resolution version using clustering
        client_coarse = create_coarse_grained(client_fine, np.array(client_locations), c_scale_factor, client_nodes)
        
        client_data.append((client_coarse, client_fine, client_nodes, client_locations))
    
    return client_data

def create_coarse_grained_matrix(fine_grained_matrix, clusters):
    n_clusters = len(np.unique(clusters))
    coarse_grained_matrix = np.zeros((n_clusters, n_clusters))
    
    # Ensure fine_grained_matrix and clusters have the same size
    valid_size = min(fine_grained_matrix.shape[0], len(clusters))
    fine_grained_matrix = fine_grained_matrix[:valid_size, :valid_size]
    clusters = clusters[:valid_size]
    
    for i in range(n_clusters):
        for j in range(n_clusters):
            mask_i = clusters == i
            mask_j = clusters == j
            coarse_grained_matrix[i, j] = np.sum(fine_grained_matrix[np.ix_(mask_i, mask_j)])
    
    return coarse_grained_matrix

def create_coarse_grained(sub_hr_matrix, locations, scale_factor, clusters):
    num_samples, height, width, channels = sub_hr_matrix.shape
    
    # Calculate number of clusters
    # n_clusters = math.ceil(height / scale_factor) * math.ceil(width / scale_factor)
    n_clusters = height // scale_factor
    # if height % scale_factor != 0:
    #     raise ValueError("Height must be divisible by scale factor")
    
    # Perform clustering
    clusters = kmeans(locations, n_clusters)
    
    # Initialize coarse-grained matrix
    coarse_height = math.floor(height / scale_factor)
    coarse_width = math.floor(width / scale_factor)
    
    # Create coarse-grained matrix
    coarse_matrix = np.array([create_coarse_grained_matrix(matrix, clusters) for matrix in sub_hr_matrix]).astype(np.float32)
    coarse_matrix = coarse_matrix.reshape(-1, n_clusters, n_clusters, channels)
    
    return coarse_matrix


#################################################################################

# Usage
# dataset = 'geant'
# scale_factor = 2
# num_clients = 4
# c_scale_factor = scale_factor  # This determines the downsampling factor for creating coarse-grained matrices for each client
# path_to_data = 'CNSM/data'
# original_size = 22
# ground_truth = f'{dataset}_original_{original_size}.npy'
# overlap_percentage = 25
# NUM_ROUNDS = 20

# train_file = f'{dataset}_coarse_{original_size//scale_factor}_x{scale_factor}.npy'
# train_set = np.load(os.path.join(path_to_data, train_file)).astype(np.float32)
# train_ground_truth = np.load(os.path.join(path_to_data, ground_truth)).astype(np.float32)
# # node_to_index = np.load(os.path.join(path_to_data, 'node_to_index.npy'), allow_pickle=True).item()

# train_set = train_set.reshape((-1, 11, 11, 1))
# train_ground_truth = train_ground_truth.reshape((-1, 22, 22, 1))

# print(f"Train set shape: {train_set.shape}", "Ground truth shape:", train_ground_truth.shape)

# train_set = preprocess(train_set)
# train_ground_truth = preprocess(train_ground_truth)

# # Extract node locations and cluster them
# nodes, locations = extract_node_locations() # for GAENT dataset

# # Create client data
# data_filename = f"client_data_{num_clients}_x{scale_factor}_overlap_{overlap_percentage}.pkl"
# client_data = create_client_data(train_ground_truth, num_clients, c_scale_factor, nodes, locations, overlap_percentage)

In [4]:
import pickle

def save_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

## Parameters 

In [5]:
dataset = 'germany'

if dataset == 'geant':
    original_size = 22
elif dataset == 'germany':
    original_size = 161

NUM_ROUNDS = 20
overlap_perc = 5
num_clients = 4
c_scale_factor = 2  # This determines the downsampling factor for creating coarse-grained matrices for each client
scale_factor = 2
path_to_data = 'CNSM/data'
ground_truth = f'{dataset}_original_{original_size}.npy'
coarse_size = original_size // scale_factor
fine_size = coarse_size * scale_factor
train_file = f'{dataset}_coarse_{coarse_size}_x{scale_factor}.npy'
coarse_size = original_size // scale_factor

In [6]:
# Load and preprocess your data
train_file = f'{dataset}_coarse_{original_size//scale_factor}_x{scale_factor}.npy'
# train_set = np.load(os.path.join(path_to_data, train_file)).astype(np.float32)
train_ground_truth = np.load(os.path.join(path_to_data, ground_truth)).astype(np.float32)
print(f"Dataset {dataset} shape:", train_ground_truth.shape)
# debug
# train_ground_truth = train_ground_truth[:400]

# train_set = train_set.reshape((-1, 11, 11, 1))
if dataset == 'geant':
    train_ground_truth = train_ground_truth.reshape((-1, 22, 22, 1))
elif dataset == 'germany':
    train_ground_truth = train_ground_truth.reshape((-1, 161, 161, 1))

# print(f"Train set shape: {train_set.shape}", "Ground truth shape:", train_ground_truth.shape)

# train_set = preprocess(train_set)
train_ground_truth = preprocess(train_ground_truth)

# Extract node locations and cluster them
nodes, locations = extract_node_locations(dataset) # for GAENT dataset

# Create client data
# n_clients = [6, 7, 8, 9, 10]
# percentages = [5, 25, 50, 75]
# for num_clients in n_clients:
#     for p in percentages:
data_filename = f"{dataset}_client_data_{num_clients}_x{scale_factor}_overlap_{overlap_perc}.pkl"
if not os.path.exists(f'CNSM/fed_data/{data_filename}'):
    print("File not found, generating client data...")
    client_data = create_client_data(train_ground_truth, num_clients, c_scale_factor, nodes, locations, overlap_perc)
    save_pickle(client_data, f'CNSM/fed_data/{data_filename}')
else:
    print("Loading data")
    client_data = load_pickle(f'CNSM/fed_data/{data_filename}')
# Print the shape of the client data
for i, client in enumerate(client_data):
    print(f"Client {i} LR:", client[0].shape, "Client {i} HR", client[1].shape)

Dataset germany shape: (8993, 161, 161)
Loading data
Client 0 LR: (8993, 23, 23, 1) Client {i} HR (8993, 46, 46, 1)
Client 1 LR: (8993, 23, 23, 1) Client {i} HR (8993, 46, 46, 1)
Client 2 LR: (8993, 23, 23, 1) Client {i} HR (8993, 46, 46, 1)
Client 3 LR: (8993, 23, 23, 1) Client {i} HR (8993, 46, 46, 1)


## Training

In [7]:
import time

# Define the TFF model
def model_fn():
    keras_model = edsr(input_depth=1, scale=2, num_filters=64, num_res_blocks=8)
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=(
            tf.TensorSpec(shape=(None, coarse_matrix_size, coarse_matrix_size, 1), dtype=tf.float32),
            tf.TensorSpec(shape=(None, fine_matrix_size, fine_matrix_size, 1), dtype=tf.float32)
        ),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
    )

def create_tf_dataset_for_client(client_data):
    def batch_format_fn(x, y):
        return (x, y)
    dataset = tf.data.Dataset.from_tensor_slices(client_data)
    dataset = dataset.shuffle(buffer_size=len(client_data[0]))
    dataset = dataset.batch(32)
    dataset = dataset.map(batch_format_fn)
    return dataset

for overlap_perc in [5, 25, 50, 75]:
    for num_clients in [3, 4, 5, 6, 7]:
        # Shapes of the client data
        if dataset == 'geant':
            data_filename = f'CNSM/fed_data/client_data_{num_clients}_x{scale_factor}_overlap_{overlap_perc}.pkl'
        elif dataset == 'germany':
            data_filename = f'CNSM/fed_data/germany_client_data_{num_clients}_x{scale_factor}_overlap_{overlap_perc}.pkl'
        if os.path.exists(data_filename):
            client_data = load_pickle(data_filename)
            coarse_matrix_size = client_data[0][0].shape[1]
            fine_matrix_size = client_data[0][1].shape[1]
        else:
            raise ValueError("The file doesn't exist. Generate client data first.")

        # Convert all clients data to float 32
        client_data = [(x[0].astype(np.float32), x[1].astype(np.float32)) for x in client_data]

        client_data = [client_data[0][0][:2000], client_data[0][1][:2000]]

        federated_train_data = [create_tf_dataset_for_client(cd) for cd in client_data]

        # Create the federated learning process
        iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
            model_fn,
            client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=1e-3),
            server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=1e-3)
        )

        # Initialize the server state
        state = iterative_process.initialize()

        #Perform federated training
        start = time.time()
        for round_num in range(NUM_ROUNDS):
            state, metrics = iterative_process.next(state, federated_train_data)
            print(f'Round {round_num}')
            print(metrics)

        end = time.time()
        print(f"Training time: {end - start}")
        # Save training time
        save_pickle(end - start, f'CNSM/fed_data/{dataset}_fed_training_time_{num_clients}_x{scale_factor}_rounds_{NUM_ROUNDS}_overlap_{overlap_perc}.pkl')
        # Save the federated weights
        fed_weights = state[0].trainable
        # Save weights
        save_pickle(fed_weights, f'CNSM/fed_data/{dataset}_fed_weights_{num_clients}_x{scale_factor}_rounds_{NUM_ROUNDS}_overlap_{overlap_perc}.pkl')

2024-09-10 20:17:05.907200: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 20:17:05.911305: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-09-10 20:17:05.911525: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-09-10 20:17:05.912280: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 20:17:05.914891: W tensorflow/core/common_runtime/gpu/gpu_d

KeyboardInterrupt: 

## Evaluation

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

train_set = np.load(os.path.join(path_to_data, train_file)).astype(np.float32)
train_ground_truth = np.load(os.path.join(path_to_data, ground_truth)).astype(np.float32)

# node_to_index = np.load(os.path.join(path_to_data, 'node_to_index.npy'), allow_pickle=True).item()

train_set = train_set.reshape((-1, coarse_size, coarse_size, 1))
train_ground_truth = train_ground_truth.reshape((-1, original_size, original_size, 1))
print(f"Train set shape: {train_set.shape}", "Ground truth shape:", train_ground_truth.shape)

train_set = preprocess(train_set)
train_ground_truth = preprocess(train_ground_truth)

# Load the federated weights
fed_weights = load_pickle(f'CNSM/fed_data/{dataset}_fed_weights_{num_clients}_x{scale_factor}_rounds_{NUM_ROUNDS}_overlap_{overlap_perc}.pkl')

# Create a new model and load the weights
model = edsr(input_depth=1, scale=2, num_filters=64, num_res_blocks=8)
model.set_weights(fed_weights)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.MeanSquaredError())

# Split the data into train and test sets
train_size = int(0.8 * len(train_set))
x_train, x_test = train_set[:train_size], train_set[train_size:]
y_train, y_test = train_ground_truth[:train_size], train_ground_truth[train_size:]

# Flatten the arrays to 1D for easier calculation
test_predictions = model.predict(x_test)

# Flatten the arrays to 1D for easier calculation
test_predictions = model.predict(x_test)
if test_predictions.shape[1] != y_test.shape[1]:
    y_test = y_test[:, :test_predictions.shape[1], :test_predictions.shape[1]] # To match the model output
y_true = y_test.flatten()
y_pred = test_predictions.flatten()

# Calculate MSE
mse = mean_squared_error(y_true, y_pred)

# Calculate RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_true, y_pred)

# Calculate MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Optional: Calculate R-squared (coefficient of determination)
from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print(f"R-squared: {r2:.4f}")

Train set shape: (8993, 80, 80, 1) Ground truth shape: (8993, 161, 161, 1)


FileNotFoundError: [Errno 2] No such file or directory: 'CNSM/fed_data/germany_fed_weights_4_x2_rounds_20_overlap_5.pkl'